***Introduction***

This document provides a detailed overview of the methods and calculations essential for converting raw sensor currents into accurate gas concentration readings using Alphasense sensors. As leaders in gas detection technology, Alphasense provides robust solutions that are crucial for a variety of applications ranging from environmental monitoring to industrial safety. The focus here is to elucidate the specific equations and algorithms that are fundamental to the sensor data interpretation process, enabling users to transform electrical signals emitted by the sensors into meaningful data.

Contained within, you will find a comprehensive exploration of the mathematical models and conversion formulas as provided by Alphasense, designed to facilitate the accurate calculation of gas concentrations from the raw currents measured by the sensors. This documentation is intended to be a valuable resource for engineers, researchers, and professionals who rely on precise and dependable gas measurement techniques. Through an in-depth understanding of these conversion principles, users can ensure the effective application of Alphasense technology in their respective fields, thereby enhancing the reliability and accuracy of their gas detection systems.

***Terms definitions***

- WEu = uncorrected raw WE output (mV - coming from BU)
- AEu = uncorrected raw AE output (mV - coming from BU)
- WEc = corrected WE output (Result Output)
- WEe = WE electronic offset on the AFE or ISB (mV - initially provided by Alphasense - source from AvSystem on runtime)
- AEe = AE electronic offset on the AFE or ISB (mV - initially provided by Alphasense - source from AvSystem on runtime)
- WEo = WE sensor zero, i.e. the sensor WE output in zero air (mV - initially provided by Alphasense - source from AvSystem on runtime)
- AEo = AE sensor zero, i.e. the sensor AE output in zero air (mV - initially provided by Alphasense - source from AvSystem on runtime)

- WET = Total WE zero offset
- AET = Total AE zero offset

- nT = temperature dependent correction factor for algorithm 1, refer to Table 3 for values
- kT = temperature dependent correction factor for algorithm 2, refer to Table 3 for values
- k'T = temperature dependent correction factor for algorithm 3, refer to Table 3 for values
- k''T = temperature dependent correction factor for algorithm 4, refer to Table 3 for values


***Algorithms***

- Algorithm 1
    ```
    WEc = (WEu - WEe) - nT * (AEu - AEe)
    ```
- Algorithm 2
    ```
    WEc = (WEu - WEe) - kT * (WEo / AEo) * (AEu - AEe)
    ```
- Algorithm 3
    ```
    WEc = (WEu - WEe) - (WEo - AEo) - k'T * (AEu - AEe)
    ```
- Algorithm 4
    ```
    WEc = (WEu - WEe) - WEo - k''T
    ```

***Sensorbee Sensors***

- CO-B4
    - Type: B
    - Recommended Algorithm: Algorithm 1

- H2S-B4
    - Type: B
    - Recommended Algorithm: 1

- NO-B4
    - Type: B
    - Recommended Algorithm: 1

- NO2-B43F
    - Type: B
    - Recommended Algorithm: 1

- OX-B431
    - Type: B
    - Recommended Algorithm: 1

- SO2-B4
    - Type: B
    - Recommended Algorithm: 4


***JavaScript Source Code Library***

In [41]:
var converter = {};
var globalParam = {};

(function(context) {
    // list of available algorithms from alphasense
    context.algorithms = {
        1: {
            parameters: ['WEu', 'WEe', 'AEu', 'AEe']
        },
        2: {
            parameters: ['WEu', 'WEe', 'WEo', 'AEo', 'AEu', 'AEe']
        },
        3: {
            parameters: ['WEu', 'WEe', 'WEo', 'AEo', 'AEu', 'AEe']
        },
        4: {
            parameters: ['WEu', 'WEe', 'WEo']
        }
    
    }
    
    // gas types definition
    context.gasTypes = {
        'CO-B4': {
            algo: 1,
            coefficients: [0.7, 0.7, 0.7, 0.7, 1, 3, 3.5, 4, 4.5]
        }, 
        'H2S-B4': {
            algo: 1,
            coefficients: [-0.6, -0.6, 0.1, 0.8, -0.7, -2.5, -2.5, -2.2, -1.8]
        }, 
        'NO-B4': {
            algo: 2,
            coefficients: [2.9, 2.9, 2.2, 1.8, 1.7, 1.6, 1.5, 1.4, 1.3]
        },
        'NO2-B43F': {
            algo: 1,
            coefficients: [1.3, 1.3, 1.3, 1.3, 1, 0.6, 0.4, 0.2, -1.5]
           
        },
        'OX-B431': {
            algo: 1,
            coefficients: [0.9, 0.9, 1, 1.3, 1.5, 1.7, 2, 2.5, 3.7]
        },
        'SO2-B4': {
            algo: 4,
            coefficients: [-4, -4, -4, -4, -4, 0, 20, 140, 450]
        }
    };
    
    // evaluate polynomial eq, where x = temperature
    context.evaluatePolynomial  = function (coeffs, x) {
        if (coeffs.length !== 9) {
            throw new Error("Invalid number of coefficients. Exactly 9 coefficients are required.");
        }
    
        let result = 0;
        coeffs.forEach((coeff, index) => {
            const power = coeffs.length - 1 - index;  
            result += coeff * Math.pow(x, power);  
        });

        return result / 100;
    };
    
    // validate if parameters needed are complete
    context.validateParameters = function(checkList, parameters) {
        const total = checkList.length;
        let ctr = 0;
        
        checkList.forEach((value, index) => {
            if (parameters[value]) {
                ctr++;
            }
        });
    
        if (ctr === total) {
            return true;
        } else {
            return false;
        }
    };
    
    // alphaSense algorithm 1
    context.algo1 = function (parameters, nT) {
        let result = 0;
        const { WEu, WEe, AEu, AEe } = parameters;
        result = (WEu - WEe) - nT * (AEu - AEe);
        return result;
    };

    // alphaSense algorithm 2
    context.algo2 = function (parameters, kT) {
        let result = 0;
        const { WEu, WEe, WEo, AEo, AEu, AEe } = parameters;
        result = (WEu - WEe) - kT * (WEo / AEo) * (AEu - AEe);
        return result;
    };

    // alphasense algorithm 3
    context.algo3 = function (parameters, kT) {
        let result = 0;
        const { WEu, WEe, WEo, AEo, AEu, AEe } = parameters;
        result = (WEu - WEe) - (WEo - AEo) - kT * (AEu - AEe);
        return result;
    };
    
    // alphaSense algorithm 4
    context.algo4 = function (parameters, kT) {
        let result = 0;
        const { WEu, WEe, WEo } = parameters;
        result = (WEu - WEe) - WEo - kT;
        return result;
    };

    // convert corrected voltage to ppm
    context.convertToPPm = function(corrected, sensitivity, offset) {
        const voltage = corrected + offset;
        const ppm = voltage / sensitivity;
        return ppm;
    };
    
    // convert to gas
    context.convertToTargetGas = function (type, temperature, voltageSensitivity, zeroVoltageOffset, parameters) {
        const { algo, coefficients } = context.gasTypes[type];
    
        if (algo === 1) {
            // algorithm 1
            const validate = context.validateParameters(context.algorithms[algo].parameters, parameters);
            if (validate) {
                const nT = context.evaluatePolynomial(coefficients, temperature);
                const WEc = context.algo1(parameters, nT);
                const ppm = context.convertToPPm(WEc, voltageSensitivity, zeroVoltageOffset);
                
                return {
                    WEc,
                    correctionFactor: nT,
                    ppm
                };
            } else {
                throw new Error("Incomplete list of parameters.");
            }
        } else if (algo === 2) {
            // algorithm 2
            const validate = context.validateParameters(context.algorithms[algo].parameters, parameters);
            if (validate) {
                const kT = context.evaluatePolynomial(coefficients, temperature);
                const WEc = context.algo2(parameters, kT);
                const ppm = context.convertToPPm(WEc, voltageSensitivity, zeroVoltageOffset);
                
                return {
                    WEc,
                    correctionFactor: kT,
                    ppm
                };
            } else {
                throw new Error("Incomplete list of parameters.");
            }
        } else if (algo === 3) {
            // algorithm 3
            const validate = context.validateParameters(context.algorithms[algo].parameters, parameters);
            if (validate) {
                const kT = context.evaluatePolynomial(coefficients, temperature);
                const WEc = context.algo3(parameters, kT);
                const ppm = context.convertToPPm(WEc, voltageSensitivity, zeroVoltageOffset);
                
                return {
                    WEc,
                    correctionFactor: kT,
                    ppm
                };
            } else {
                throw new Error("Incomplete list of parameters.");
            }
        } else if (algo === 4) {
            // algorithm 4 
            const validate = context.validateParameters(context.algorithms[algo].parameters, parameters);
            if (validate) {
                const kT = context.evaluatePolynomial(coefficients, temperature);
                const WEc = context.algo4(parameters, kT);
                const ppm = context.convertToPPm(WEc, voltageSensitivity, zeroVoltageOffset);
                
                return {
                    WEc,
                    correctionFactor: kT,
                    ppm
                };
            } else {
                throw new Error("Incomplete list of parameters.");
            }
        }
    };
})(converter);


***Arguments***

Enter Your Parameters below:

In [44]:
globalParam.WEu = 0.9847094416618347;
globalParam.WEe = 287;
globalParam.AEu = 0.9443225264549255;
globalParam.AEe = 299;
globalParam.AEo = 248;
globalParam.WEo = 269;

globalParam.sensor_type = 'CO-B4';
globalParam.temperature = 0;
globalParam.sensitivity = -494.8223;
globalParam.zero_offset = -0.0657451698113208;

-0.0657451698113208


***Test Environment***

In [47]:
var result = converter.convertToTargetGas(globalParam.sensor_type, globalParam.temperature, globalParam.sensitivity, globalParam.zero_offset, {
    'WEu': globalParam.WEu, 
    'WEe': globalParam.WEe, 
    'AEu': globalParam.AEu, 
    'AEe': globalParam.AEe,
    'AEo': globalParam.AEo,
    'WEo': globalParam.WEo
});

console.log('result', result);

result {
  WEc: -272.6027850720286,
  correctionFactor: 0.045,
  ppm: 0.5510433346311189
}
